In [1]:
data_path = "../data/70-15-15"
model_path = "../models"

In [2]:
!pip install tensorflow
#!pip install datasets

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

import numpy as np
import pandas as pd



In [4]:
treino=pd.read_csv(data_path+'/comunic_treino.csv', encoding='utf-8')
teste=pd.read_csv(data_path+'/comunic_teste.csv', encoding='utf-8')
validacao=pd.read_csv(data_path+'/comunic_valid.csv', encoding='utf-8')

In [5]:
def TFIDF(X_train, X_test,MAX_NB_WORDS=4000):
    vectorizer_x = TfidfVectorizer(max_features=MAX_NB_WORDS)
    X_train = vectorizer_x.fit_transform(X_train).toarray()
    X_test = vectorizer_x.transform(X_test).toarray()
    print("tf-idf with",str(np.array(X_train).shape[1]),"features")
    return (X_train,X_test)

In [6]:
def make_tfidf(coluna):
    vectorizer = TfidfVectorizer()
    vectors = np.asarray(vectorizer.fit_transform(coluna).todense())
    return vectors

In [7]:
X_train, X_test = TFIDF(treino["text"], teste["text"])
Y_train=treino.label
Y_test=teste.label
X_val=make_tfidf(validacao.text)
Y_val=validacao.label

tf-idf with 4000 features


# Definição do modelo de classificacao DNN

In [8]:
from tensorflow.python.keras import models
from tensorflow.python.keras import layers
from tensorflow.python.keras import losses
from tensorflow.python.keras import optimizers

from tensorflow.python.keras.callbacks import EarlyStopping

early_stopping =  EarlyStopping(
    monitor='val_accuracy', 
    patience=4
)

def Build_Model_DNN_Text(shape, nClasses, dropout=0.4):
    model = models.Sequential()
    model.add(layers.Dense(4000,input_dim=shape,activation='relu'))
    model.add(layers.Dense(shape,activation='relu'))
    model.add(layers.Dropout(dropout))
    model.add(layers.Dense(1024,activation='relu'))
    model.add(layers.Dropout(dropout))
    model.add(layers.Dense(128,activation='relu'))
    model.add(layers.Dropout(dropout))    
    model.add(layers.Dense(1024,activation='relu'))
    model.add(layers.Dropout(dropout))    
    model.add(layers.Dense(nClasses, activation='sigmoid'))
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='Adam',
                  metrics=['accuracy'])
    model.summary()
    return model

2023-03-09 13:46:41.441179: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-09 13:46:41.998540: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-09 13:46:41.998595: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-09 13:46:41.998602: W tensorfl

In [9]:
from sklearn import metrics
model_DNN = Build_Model_DNN_Text(X_train.shape[1], 13)
model_DNN.fit(X_train, Y_train,
                              validation_data=(X_test, Y_test),
                              epochs=50,
                              batch_size=128,
                              verbose=2)
                              #callbacks=[early_stopping])
predicted = model_DNN.predict(X_test)
Y_pred = np.argmax(predicted, axis=1)
print(metrics.classification_report(Y_test, Y_pred))

2023-03-09 13:46:42.509108: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-09 13:46:42.518341: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-09 13:46:42.518610: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-09 13:46:42.519274: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

ResourceExhaustedError: {{function_node __wrapped__RandomUniform_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[4000,4000] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:RandomUniform]

### Métricas

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, precision_score, recall_score, hamming_loss
precision = precision_score(Y_test, Y_pred,average='micro')
recall = recall_score(Y_test, Y_pred,average='micro')
f1_micro_average = f1_score(y_true=Y_test, y_pred=Y_pred, average='micro')
#roc_auc = roc_auc_score(Y_test, Y_pred, multi_class ='ovr', average = 'micro')
accuracy = accuracy_score(Y_test, Y_pred)
hloss = hamming_loss(Y_test, Y_pred)

In [ ]:
print("Acurácia:",accuracy)
print("Precisão:",precision)
print("Recall:",recall)
print("F1:",f1_micro_average)
#print("Roc_auc:",roc_auc)
print("Hamming Loss:",hloss)